# Install Libraries

In [ ]:
!pip install datasets umap-learn hdbscan

# About Dataset

In this tutorial, we'll be working with the ArXiv articles dataset. ArXiv is a free platform that hosts scholarly papers, mostly in fields like computer science, math, and physics. We'll focus on articles from the "Computation and Language" section, which includes around 44,000 abstracts!

In [ ]:
# Import the load_dataset function from the datasets library
from datasets import load_dataset

# Load the "arxiv_nlp" dataset from Hugging Face, specifically the "train" split
dataset = load_dataset("maartengr/arxiv_nlp")["train"] # We will use train datasplit

# Extract the "Abstracts" column from the dataset
abstracts = dataset["Abstracts"]

# Extract the "Titles" column from the dataset
titles = dataset["Titles"]

# Lets Implement Text Clustering Pipeline Step by Step

# Step 1: Convert Texts to Embeddings

In [ ]:
# Import the SentenceTransformer class from the sentence_transformers library
from sentence_transformers import SentenceTransformer

# Initialize a pre-trained sentence embedding model ('thenlper/gte-small')
embedding_model = SentenceTransformer('thenlper/gte-small')

# Encode the abstracts into vector embeddings, with a progress bar shown
document_embeddings = embedding_model.encode(abstracts, show_progress_bar=True)


In [ ]:
document_embeddings

In [ ]:
document_embeddings.shape

# Step 2: Reduce Dimensionality

In [ ]:
# Import the UMAP class from the umap library for dimensionality reduction
from umap import UMAP

# Initialize the UMAP model to reduce embeddings to 5 dimensions
# n_components=5: Target dimensionality (5 dimensions)
# min_dist=0.0: Minimum distance between points in the low-dimensional space
# metric='cosine': Use cosine similarity as the distance metric
# random_state=42: Set a fixed random seed for reproducibility
umap_model_object = UMAP(
    n_components=5, min_dist=0.0, metric='cosine', random_state=42
)

# Apply UMAP to reduce the dimensionality of the document embeddings
new_compressed_embeddings = umap_model_object.fit_transform(document_embeddings)


# Step 3: Apply Clustering

We'll use the UMAP algorithm because it does a better job at capturing complex, nonlinear relationships and structures compared to PCA.

In [ ]:
# Import the HDBSCAN class from the hdbscan library for clustering
from hdbscan import HDBSCAN

# Initialize the HDBSCAN model with specified parameters:
# min_cluster_size=50: Minimum size of clusters to be considered
# metric='euclidean': Use Euclidean distance to measure similarity between points
# cluster_selection_method='eom': Select clusters based on the Excess of Mass method
# Fit the model on the reduced embeddings (new_compressed_embeddings)
hdbscan_model = HDBSCAN(
    min_cluster_size=50, metric='euclidean', cluster_selection_method='eom'
).fit(new_compressed_embeddings)

# Extract the cluster labels assigned to each data point
clusters = hdbscan_model.labels_

# Count how many unique clusters were generated
len(set(clusters))


# Lets Inspect Clusters

In [ ]:
# Import the numpy library for numerical operations
import numpy as np

# Set the target cluster to 0
cluster = 0

# Loop through the first three document indices in cluster 0
# np.where(clusters == cluster)[0] returns the indices of documents in the specified cluster
# [:3] selects the first three documents
for index in np.where(clusters == cluster)[0][:3]:
    # Print the first 100 characters of each abstract, followed by ellipsis and a newline
    print(abstracts[index][:100] + "... \n")


In [ ]:
# Import the numpy library for numerical operations
import numpy as np

# Set the target cluster to 2
cluster = 2

# Loop through the first three document indices in cluster 2
# np.where(clusters == cluster)[0] returns the indices of documents in cluster 2
# [:3] selects the first three documents from those indices
for index in np.where(clusters == cluster)[0][:3]:
    # Print the first 100 characters of each abstract, followed by ellipsis and a newline
    print(abstracts[index][:100] + "... \n")
